In [ ]:
import json
import re
from datetime import datetime

class CitationLogger:
    def __init__(self, author, title, repo_url, year=None):
        self.author = author
        self.title = title
        self.repo_url = repo_url
        self.year = year or datetime.now().year
        self.bib_key = self._generate_bib_key()

    def _generate_bib_key(self):
        first = re.sub(r'[^a-zA-Z]', '', self.author.split()[0].lower())
        title_token = re.sub(r'[^a-zA-Z]', '', self.title.split()[0].lower())
        return f"{first}{self.year}{title_token}"

    def markdown(self):
        return f"""### Citation  
{self.author} ({self.year}). *{self.title}*. GitHub Repository: [{self.repo_url.split('/')[-1]}]({self.repo_url})
"""

    def bibtex(self):
        return f"""@misc{{{self.bib_key},
  author       = {{{self.author}}},
  title        = {{{self.title}}},
  year         = {{{self.year}}},
  howpublished = {{\\url{{{self.repo_url}}}}},
  note         = {{GitHub Repository}}
}}"""

    def json(self):
        return {
            "author": self.author,
            "year": self.year,
            "title": self.title,
            "repository": "GitHub",
            "url": self.repo_url
        }

    def display(self):
        print(" Markdown Citation:\n")
        print(self.markdown())
        print("\n BibTeX Citation:\n")
        print(self.bibtex())
        print("\n JSON Citation:\n")
        print(json.dumps(self.json(), indent=4))

citation = CitationLogger(
    author="Nouri Baher",
    title="Hybrid Oversampling for Intrusion Detection: SMOTE + KGSMOTE",
    repo_url="https://github.com/Nouribaher/ids-hybrid-oversampling-smote-kgsmote",
    year=2025
)

citation.display()

In [1]:
# Classifier - Dense Neural Network for Attack Prediction

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

#  Load Compressed Features and Labels

X_df = pd.read_excel("compressed-Hybrid_Balanced-Denoising.xlsx", sheet_name="Compressed_Features", engine="openpyxl")
y_df = pd.read_excel("results-Hybrid_Balanced.xlsx", engine="openpyxl")
y_raw = y_df['label_name'].values


#  Encode Labels

le = LabelEncoder()
y = le.fit_transform(y_raw)
class_names = list(le.classes_)


#  Prepare Feature Matrix

X = X_df.select_dtypes(include=[np.number]).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#  Build Dense Neural Network

def build_classifier(input_dim, n_classes):
    inputs = Input(shape=(input_dim,))
    x = Dense(64, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

input_dim = X_train.shape[1]
n_classes = len(class_names)
model = build_classifier(input_dim, n_classes)

#  Train Model

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.fit(X_train, y_train,
          epochs=100,
          batch_size=64,
          validation_split=0.1,
          callbacks=[early_stop],
          verbose=2)


# Evaluate Model

y_pred = np.argmax(model.predict(X_test), axis=1)
acc = accuracy_score(y_test, y_pred)
print(f"\n NOURI BAHR --- Accuracy: {acc:.4f}")

# Classification Report
report_dict = classification_report(y_test, y_pred, target_names=class_names, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_excel("Classification_Report_DNN.xlsx")
print(" Classification report saved to 'Classification_Report_DNN.xlsx'")

# Confusion Matrix
cm = pd.DataFrame(confusion_matrix(y_test, y_pred),
                  index=class_names,
                  columns=class_names)
cm.to_excel("Confusion_Matrix_DNN.xlsx")
print("Confusion matrix saved to 'Confusion_Matrix_DNN.xlsx'")

# Save Model
model.save("DAE_Neural_Classifier.h5")
print("Model saved as 'DAE_Neural_Classifier.h5'")

Epoch 1/100
791/791 - 3s - 4ms/step - accuracy: 0.4581 - loss: 1.7940 - val_accuracy: 0.8332 - val_loss: 0.6873
Epoch 2/100
791/791 - 2s - 2ms/step - accuracy: 0.7203 - loss: 0.8312 - val_accuracy: 0.8859 - val_loss: 0.4253
Epoch 3/100
791/791 - 2s - 3ms/step - accuracy: 0.7837 - loss: 0.6418 - val_accuracy: 0.9078 - val_loss: 0.3416
Epoch 4/100
791/791 - 2s - 2ms/step - accuracy: 0.8156 - loss: 0.5512 - val_accuracy: 0.9187 - val_loss: 0.2929
Epoch 5/100
791/791 - 2s - 2ms/step - accuracy: 0.8357 - loss: 0.4877 - val_accuracy: 0.9247 - val_loss: 0.2426
Epoch 6/100
791/791 - 2s - 2ms/step - accuracy: 0.8549 - loss: 0.4380 - val_accuracy: 0.9427 - val_loss: 0.2148
Epoch 7/100
791/791 - 2s - 2ms/step - accuracy: 0.8676 - loss: 0.4009 - val_accuracy: 0.9468 - val_loss: 0.1877
Epoch 8/100
791/791 - 2s - 3ms/step - accuracy: 0.8785 - loss: 0.3714 - val_accuracy: 0.9519 - val_loss: 0.1643
Epoch 9/100
791/791 - 3s - 3ms/step - accuracy: 0.8875 - loss: 0.3444 - val_accuracy: 0.9532 - val_loss:

 Classification report saved to 'Classification_Report_DNN.xlsx'
Confusion matrix saved to 'Confusion_Matrix_DNN.xlsx'
Model saved as 'DAE_Neural_Classifier.h5'


In [4]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("DAE_Neural_Classifier.h5")

# View model architecture
model.summary()

# Predict on new data (example)
# X_new should be preprocessed the same way as training data
# y_pred = model.predict(X_new)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 38)                  │           1,254 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,448 (21.29 KB)

 Trainable params: 5,446 (21.27 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)